<a href="https://colab.research.google.com/github/dsmolinski-bluerank/Auto_Semantic_Site_Maps/blob/main/Tworzenie_mapy_tematycznej_przez_Nomic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalacja biblioteki Nomic & ustawienie klucza API.

In [ ]:
!pip install nomic
import pandas as pd
import json
import nomic
from nomic import atlas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.7 MB/s eta 0:00:00
  Created wheel for nomic: filename=nomic-3.0.35-py3-none-any.whl size=44364 sha256=dfb04a758f2ea8a1efc94354b95743add711c3f4c307867c4b5cbdfaaeed9765
  Stored in directory: /root/.cache/pip/wheels/35/47/bf/bd1ad7c376e204b277fb13e8e09921a1e6b19b20dd42ee2c0d
Successfully built nomic


Klucz API Nomic. Wklei swój pomiędzy ''. Klucz można wygenerować w zakładce API Keys.

In [ ]:
nomic.login('nk-ekec4mB-6qzrEbEhF4UzG6lImVo600rxHKEGzW_e5UA')  # Tu wklej swój klucz API z usługi nomic.ai

Skrypt do generowania mapy embeddingowej

In [ ]:
def main():
    # Załaduj dane z pliku JSON
    with open('/content/api_results_partial (8).json', 'r') as file:
        data = json.load(file)

    # Tworzenie DataFrame z danymi z pliku JSON
    results_df = pd.json_normalize(data)

    # Debugowanie: Wyświetlanie wszystkich kolumn w DataFrame
    print("Kolumny w results_df:", results_df.columns)

    # Wyodrębnianie wymaganych kolumn
    results_df['title'] = results_df['data.data.title']
    results_df['content'] = results_df['data.data.content']
    results_df['url'] = results_df['url']

    required_columns = {'url', 'title', 'content'}

    # Sprawdzenie, czy wszystkie wymagane kolumny są obecne w DataFrame
    if required_columns.issubset(results_df.columns):
        filtered_df = results_df[['url', 'title', 'content']]

        if not filtered_df.empty:
            # Sprawdzenie struktury DataFrame
            print(filtered_df.head())

            # Łączenie kolumn 'url', 'title' i 'content' w jedną kolumnę 'indexed_field'
            filtered_df['indexed_field'] = filtered_df.apply(
                lambda row: f"{row['url']} {row['title']} {row['content']}", axis=1
            )

            # Przekazanie danych do atlas.map_data, używając nowo utworzonej kolumny 'indexed_field'
            dataset = atlas.map_data(data=filtered_df, indexed_field='indexed_field')
            print(dataset)
        else:
            print("Brak danych do przetworzenia.")
    else:
        missing_columns = required_columns - set(results_df.columns)
        print(f"Brakujące kolumny: {missing_columns}")

if __name__ == "__main__":
    main()


<ipython-input-4-25fad4ff935d>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['indexed_field'] = filtered_df.apply(
/usr/local/lib/python3.10/dist-packages/nomic/atlas.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[id_field] = [b64int(i) for i in range(data.shape[0])]
2024-07-04 12:26:28.487 | WARNING  | nomic.atlas:map_data:98 - An ID field was not specified in your data so one was generated for you in insertion order.


Kolumny w results_df: Index(['index', 'url', 'status', 'data.code', 'data.status', 'data.data.title',
       'data.data.url', 'data.data.content', 'data.data.publishedTime',
       'data'],
      dtype='object')
                                                 url  \
0                    https://www.senuto.com/pl/blog/   
1  https://www.senuto.com/pl/blog/jak-poprawic-se...   
2  https://www.senuto.com/pl/blog/analiza-konkure...   
3     https://www.senuto.com/pl/blog/slowa-kluczowe/   
4  https://www.senuto.com/pl/blog/co-to-jest-digi...   

                                               title  \
0   Blog o SEO, pozycjonowaniu i marketingu | Senuto   
1  Jak poprawić SEO strony, wyciskając więcej z W...   
2  Analiza konkurencji – jak zdobyć przewagę na r...   
3  Słowa kluczowe w SEO – gdzie ich szukać i jak ...   
4                      Co to jest digital marketing?   

                                             content  
0  🔥 Zgarnij podwójną premię: dodatkowe 10% rabat...  
1  J

2024-07-04 12:26:30.559 | INFO     | nomic.dataset:_create_project:850 - Creating dataset `dynamic-lecun`
2024-07-04 12:26:30.926 | INFO     | nomic.atlas:map_data:110 - Uploading data to Atlas.
2024-07-04 12:26:30.973 | WARNING  | nomic.dataset:_validate_and_correct_arrow_upload:310 - Replacing 361 null values for field title with string 'null'. This behavior will change in a future version.
2024-07-04 12:26:30.980 | WARNING  | nomic.dataset:_validate_and_correct_arrow_upload:310 - Replacing 361 null values for field content with string 'null'. This behavior will change in a future version.
1it [00:01,  1.03s/it]
2024-07-04 12:26:32.032 | INFO     | nomic.dataset:_add_data:1529 - Upload succeeded.
2024-07-04 12:26:32.036 | INFO     | nomic.atlas:map_data:125 - `dsalkowski/dynamic-lecun`: Data upload succeeded to dataset`
2024-07-04 12:26:33.914 | INFO     | nomic.dataset:create_index:1233 - Created map `dynamic-lecun` in dataset `dsalkowski/dynamic-lecun`: https://atlas.nomic.ai/data/

dynamic-lecun: https://atlas.nomic.ai/data/dsalkowski/dynamic-lecun
